En este codigo utilizamos los embeddings ya calculados de Bert para calcular las distancias.

Podemos utilizar cualquier grupo de celdas individualmente

# 1. Dataset completo

In [34]:
import pandas as pd

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."


In [37]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [38]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [39]:
import time
import numpy as np

In [40]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [41]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [42]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         14100
Chimu          4791
Wari           4712
Nasca          2994
Sican          1333
Chancay         916
Cajamarca       882
Tiahuanaco      699
Salinar         669
Cupisnique      522
Vicus           363
Recuay          327
Name: culture_cl, dtype: int64

In [43]:
seconds_per_data = 0.000082850766389
distances = {p: list() for p in culture_pairs}

for i in range(0, len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.embeddingCLS
    embed_c2 = df_c2.embeddingCLS

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_completo.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

0 -> Cultures: Sican, Recuay
Estimated time in seconds: 36.11
Estimated time in minutes: 01:36.11


KeyboardInterrupt: 

# 1.1 Por periodo

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."


In [4]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [5]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [6]:
import time
import numpy as np

In [7]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [8]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [9]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         14100
Chimu          4791
Wari           4712
Nasca          2994
Sican          1333
Chancay         916
Cajamarca       882
Tiahuanaco      699
Salinar         669
Cupisnique      522
Vicus           363
Recuay          327
Name: culture_cl, dtype: int64

In [10]:
ctimes = df.cronology_time.unique().tolist()
ctimes

[7, 6, 5, 8]

In [11]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.embeddingCLS
        embed_c2 = df_c2.embeddingCLS

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_completo_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 7


culture_cl
Wari          4712
Sican         1333
Tiahuanaco     699
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Sican, Wari


KeyboardInterrupt: 

# 2. Dataset Menos 1

In [44]:
import pandas as pd

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."


In [47]:
df = df.loc[df['numero_palabras'] != 1]

In [48]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [49]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [50]:
import time
import numpy as np

In [51]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [52]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [53]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13637
Chimu          4594
Wari           4241
Nasca          2963
Sican          1331
Chancay         697
Cajamarca       686
Tiahuanaco      653
Salinar         587
Cupisnique      358
Vicus           355
Recuay          308
Name: culture_cl, dtype: int64

In [54]:
seconds_per_data = 0.000082850766389
distances = {p: list() for p in culture_pairs}

for i in range(0, len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.embeddingCLS
    embed_c2 = df_c2.embeddingCLS

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_menos_uno.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

0 -> Cultures: Sican, Recuay
Estimated time in seconds: 33.96
Estimated time in minutes: 01:33.96


KeyboardInterrupt: 

# 2.1 Por periodo

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

In [ ]:
df = df.loc[df['numero_palabras'] != 1]

In [ ]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [ ]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [ ]:
import time
import numpy as np

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [ ]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13637
Chimu          4594
Wari           4241
Nasca          2963
Sican          1331
Chancay         697
Cajamarca       686
Tiahuanaco      653
Salinar         587
Cupisnique      358
Vicus           355
Recuay          308
Name: culture_cl, dtype: int64

In [ ]:
ctimes = df.cronology_time.unique().tolist()
ctimes

[7, 6, 5, 8]

In [ ]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.embeddingCLS
        embed_c2 = df_c2.embeddingCLS

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_menos_uno_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 7


culture_cl
Wari          4241
Sican         1331
Tiahuanaco     653
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Sican, Wari
Time distances between: 2081.9039072990417s
Mean distance: 0.31695677213225587

1 -> Cultures: Sican, Tiahuanaco
Time distances between: 322.7676362991333s
Mean distance: 0.3205893272761752

2 -> Cultures: Wari, Tiahuanaco
Time distances between: 1022.2446486949921s
Mean distance: 0.32222572684162154

Cronology_time: 6


culture_cl
Moche        13637
Nasca         2963
Cajamarca      686
Recuay         308
Name: culture_cl, dtype: int64

Culture pairs: 6
0 -> Cultures: Recuay, Moche
Time distances between: 1576.441192150116s
Mean distance: 0.32286449734699957

1 -> Cultures: Recuay, Cajamarca
Time distances between: 76.56543612480164s
Mean distance: 0.3230139212208876

2 -> Cultures: Recuay, Nasca
Time distances between: 330.11481976509094s
Mean distance: 0.3244304350847038

3 -> Cultures: Moche, Cajamarca
Time distances between: 3426.2054176330566s
Mean distance: 0.31448792418329174

4 -> Cultures: Moche, Nasca
Time distances between: 14722.26720237732s
Mean distance: 0.3159564283108112

5 -> Cultures: Cajamarca, Nasca
Time distances between: 737.9977324008942s
Mean distance: 0.31603973898505455

Cronology_time: 5


culture_cl
Salinar       587
Cupisnique    358
Vicus         355
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Cupisnique, Vicus
Time distances between: 50.723666191101074s
Mean distance: 0.31829341948256884

1 -> Cultures: Cupisnique, Salinar
Time distances between: 76.14084506034851s
Mean distance: 0.32190904686252986

2 -> Cultures: Vicus, Salinar
Time distances between: 76.45954751968384s
Mean distance: 0.3219268658522713

Cronology_time: 8


culture_cl
Chimu      4594
Chancay     697
Name: culture_cl, dtype: int64

Culture pairs: 1
0 -> Cultures: Chimu, Chancay
Time distances between: 1164.6084773540497s
Mean distance: 0.3172671097538279



# 3. Dataset Menos 2

In [55]:
import pandas as pd

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."


In [58]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]

In [59]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [60]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [61]:
import time
import numpy as np

In [62]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [63]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [64]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13415
Chimu          4546
Wari           4145
Nasca          2959
Sican          1331
Chancay         685
Tiahuanaco      653
Salinar         582
Cajamarca       520
Vicus           355
Cupisnique      354
Recuay          302
Name: culture_cl, dtype: int64

In [65]:
seconds_per_data = 0.000082850766389
distances = {p: list() for p in culture_pairs}

for i in range(0, len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.embeddingCLS
    embed_c2 = df_c2.embeddingCLS

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_menos_dos.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

0 -> Cultures: Sican, Recuay
Estimated time in seconds: 33.3
Estimated time in minutes: 01:33.3


KeyboardInterrupt: 

# 3.1 Por periodo

In [12]:
import pandas as pd

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."


In [15]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]

In [16]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [17]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [18]:
import time
import numpy as np

In [19]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [25]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [26]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13415
Chimu          4546
Wari           4145
Nasca          2959
Sican          1331
Chancay         685
Tiahuanaco      653
Salinar         582
Cajamarca       520
Vicus           355
Cupisnique      354
Recuay          302
Name: culture_cl, dtype: int64

In [27]:
ctimes = df.cronology_time.unique().tolist()
ctimes = [5, 7, 8, 6]

In [33]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.embeddingCLS
        embed_c2 = df_c2.embeddingCLS

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_menos_dos_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 6


culture_cl
Moche        13415
Nasca         2959
Cajamarca      520
Recuay         302
Name: culture_cl, dtype: int64

Culture pairs: 6
4 -> Cultures: Moche, Nasca
Time distances between: 10807.102430820465s
Mean distance: 0.31601822222878617

5 -> Cultures: Cajamarca, Nasca
Time distances between: 418.844144821167s
Mean distance: 0.3138001162697261



# 4. Dataset Menos 3

In [66]:
import pandas as pd

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."


In [69]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]
df = df.loc[df['numero_palabras'] != 3]

In [70]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [71]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [72]:
import time
import numpy as np

In [73]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [74]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [75]:
df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False)

culture_cl
Moche         13207
Chimu          4497
Wari           4025
Nasca          2428
Sican          1329
Chancay         661
Tiahuanaco      616
Salinar         576
Cajamarca       514
Vicus           355
Cupisnique      340
Recuay          296
Name: culture_cl, dtype: int64

In [76]:
seconds_per_data = 0.000082850766389
distances = {p: list() for p in culture_pairs}

for i in range(0, len(culture_pairs)):
    start_time = time.time()
    c1 = culture_pairs[i][0]
    c2 = culture_pairs[i][1]

    print(f'{i} -> Cultures: {c1}, {c2}')
    df_c1 = df[df.culture_cl == c1]
    df_c2 = df[df.culture_cl == c2]

    # Time estimation
    len_c1 = len(df_c1)
    len_c2 = len(df_c2)
    total_data = len_c1 * len_c2
    estimated_s = total_data * seconds_per_data
    estimated_minute = '0' + str(round(estimated_s / 60)) if round(estimated_s / 60) < 10 else str(round(estimated_s / 60))
    estimated_second = '0' + str(np.round(estimated_s % 60, 2)) if np.round(estimated_s % 60, 2) < 10 else str(np.round(estimated_s % 60, 2))
    print(f'Estimated time in seconds: {np.round(estimated_s, 2)}')
    print(f'Estimated time in minutes: {estimated_minute}:{estimated_second}')

    embed_c1 = df_c1.embeddingCLS
    embed_c2 = df_c2.embeddingCLS

    for e1 in embed_c1:
        for e2 in embed_c2:
            dist = 1 - _cosine_similarity(e1, e2)
            distances[(c1, c2)].append(dist)

    end_time = time.time()

    # Save distances in file
    mean = np.mean(distances[(c1, c2)])
    file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_menos_tres.txt', 'a')
    file.write(f'{i}: {c1}-{c2} -> {mean}\n')
    file.close()

    print(f'Time distances between: {end_time - start_time}s')
    print(f'Mean distance: {mean}')
    print()

0 -> Cultures: Sican, Recuay
Estimated time in seconds: 32.59
Estimated time in minutes: 01:32.59


KeyboardInterrupt: 

# 4.1 Por periodo

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/peru_larco/Data/base_n150/base_n150_bert.csv', sep=';')
df.head()

,Unnamed: 0,catalogation_id,cronology,cronology_time,culture_cl,morfofunctional_category,description,principal_scene,decoration_tecnique_external_body_section1,color_external_body_section1,...,trait_n103,trait_n104,trait_n105,file_path,image_path,description_length,tokens,numero_palabras,embeddingCLS,embeddingMean
0,0,ML020107,Horizonte Medio,7,Sican,botella doble cuerpo asa puente cintada silbadora,botella doble cuerpo asa puente cintada silbad...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020107a.jpg,191,[CLS] botella doble cuerpo asa puente cinta ##...,21,"[0.07349269092082977, -0.7621917128562927, 0.0...","[0.0001518863718956709, -0.38347482681274414, ..."
1,1,ML020108,Horizonte Medio,7,Sican,botella doble pico asa puente cintada escultorica,botella doble pico asa puente cintada escultor...,NaN,pintado escultorico,rojo y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020108a.jpg,245,[CLS] botella doble pico asa puente cinta ##da...,29,"[-0.16221104562282562, -0.5011821985244751, 0....","[-0.012210816144943237, -0.4184707999229431, -..."
2,2,ML020109,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020109a.jpg,134,[CLS] botella gol ##let ##e asa puente cinta #...,16,"[-0.3746039569377899, -0.3084278702735901, -0....","[-0.24044576287269592, 0.03168115019798279, -0..."
3,3,ML020110,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y naranja,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020110a.jpg,125,[CLS] botella gol ##let ##e asa puente cinta #...,15,"[0.22687536478042603, -0.7911874651908875, -0....","[0.03867929428815842, -1.0952448844909668, -0...."
4,4,ML020111,Horizonte Medio,7,Sican,botella gollete asa puente cintada protoma sil...,botella gollete asa puente cintada protoma sil...,NaN,pintado escultorico,crema y marron,...,0,1,0,data/sican_7/7 ADMINISTRADOR COLECCIONES VIRTU...,data/images/ML020111a.jpg,158,[CLS] botella gol ##let ##e asa puente cinta #...,18,"[-0.1266753375530243, -0.5457064509391785, 0.3...","[0.10204872488975525, -0.17494848370552063, -0..."


In [ ]:
df = df.loc[df['numero_palabras'] != 1]
df = df.loc[df['numero_palabras'] != 2]
df = df.loc[df['numero_palabras'] != 3]

In [ ]:
def str_to_list(string):
    string_list = string.strip('][').split(', ')
    float_list = [float(i) if len(i) > 0 else 0.0 for i in string_list]
    return float_list

In [ ]:
df.embeddingCLS = df.embeddingCLS.apply(str_to_list)
df.embeddingMean = df.embeddingMean.apply(str_to_list)

In [ ]:
import time
import numpy as np

In [ ]:
def get_positive_positions(v1, v2):
    positions = [i for i in range(len(df.embeddingCLS[0])) if v1[i] != 0.0 and v2[i] != 0.0]
    return positions

def _cosine_similarity(v1, v2):
    positions = get_positive_positions(v1, v2)
    if len(positions) == 0:
        return 0
    else:
      dot_product = np.dot(v1, v2)
      norm_vector1 = np.linalg.norm(v1)
      norm_vector2 = np.linalg.norm(v2)

      similarity = dot_product / (norm_vector1 * norm_vector2)
      return similarity

In [ ]:
cultures = df.culture_cl.unique().tolist()
culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
len(culture_pairs)

66

In [ ]:
ctimes = df.cronology_time.unique().tolist()
ctimes

[7, 6, 5, 8]

In [11]:
for ctime in ctimes:
    filtered_df = df[df['cronology_time'] == ctime]
    print(f'Cronology_time: {ctime}')
    display(filtered_df.groupby('culture_cl')['culture_cl'].count().sort_values(ascending=False))

    cultures = filtered_df.culture_cl.unique().tolist()
    culture_pairs = [(a, b) for idx, a in enumerate(cultures) for b in cultures[idx + 1:]]
    print(f'Culture pairs: {len(culture_pairs)}')

    distances = {p: list() for p in culture_pairs}

    for i in range(0, len(culture_pairs)):
        start_time = time.time()
        c1 = culture_pairs[i][0]
        c2 = culture_pairs[i][1]

        print(f'{i} -> Cultures: {c1}, {c2}')
        df_c1 = filtered_df[filtered_df.culture_cl == c1]
        df_c2 = filtered_df[filtered_df.culture_cl == c2]

        embed_c1 = df_c1.embeddingCLS
        embed_c2 = df_c2.embeddingCLS

        for e1 in embed_c1:
            for e2 in embed_c2:
                dist = 1 - _cosine_similarity(e1, e2)
                distances[(c1, c2)].append(dist)

        end_time = time.time()

        # Save distances in file
        mean = np.mean(distances[(c1, c2)])
        file = open(f'/content/drive/MyDrive/peru_larco/1. Distances/Results/base_n150_bert_dataset_menos_tres_ctime_{ctime}.txt', 'a')
        file.write(f'{i}: {c1}-{c2} -> {mean}\n')
        file.close()

        print(f'Time distances between: {end_time - start_time}s')
        print(f'Mean distance: {mean}')
        print()

Cronology_time: 7


culture_cl
Wari          4025
Sican         1329
Tiahuanaco     616
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Sican, Wari
Time distances between: 1334.1868045330048s
Mean distance: 0.31752077926777295

1 -> Cultures: Sican, Tiahuanaco
Time distances between: 202.84290170669556s
Mean distance: 0.3203302334626925

2 -> Cultures: Wari, Tiahuanaco
Time distances between: 618.2722885608673s
Mean distance: 0.3223881249701216

Cronology_time: 6


culture_cl
Moche        13207
Nasca         2428
Cajamarca      514
Recuay         296
Name: culture_cl, dtype: int64

Culture pairs: 6
0 -> Cultures: Recuay, Moche
Time distances between: 974.3449547290802s
Mean distance: 0.32320790285277934

1 -> Cultures: Recuay, Cajamarca
Time distances between: 38.46380805969238s
Mean distance: 0.32081553444356675

2 -> Cultures: Recuay, Nasca
Time distances between: 180.99397087097168s
Mean distance: 0.3240806996218901

3 -> Cultures: Moche, Cajamarca
Time distances between: 1695.088632106781s
Mean distance: 0.31175807704108516

4 -> Cultures: Moche, Nasca
Time distances between: 7495.194676160812s
Mean distance: 0.31524491564732426

5 -> Cultures: Cajamarca, Nasca
Time distances between: 279.92381739616394s
Mean distance: 0.31256453714943516

Cronology_time: 5


culture_cl
Salinar       576
Vicus         355
Cupisnique    340
Name: culture_cl, dtype: int64

Culture pairs: 3
0 -> Cultures: Cupisnique, Vicus
Time distances between: 31.38450002670288s
Mean distance: 0.32041373445826304

1 -> Cultures: Cupisnique, Salinar
Time distances between: 44.299739599227905s
Mean distance: 0.32359723455211253

2 -> Cultures: Vicus, Salinar
Time distances between: 46.08865690231323s
Mean distance: 0.3215538311900831

Cronology_time: 8


culture_cl
Chimu      4497
Chancay     661
Name: culture_cl, dtype: int64

Culture pairs: 1
0 -> Cultures: Chimu, Chancay
Time distances between: 667.5403945446014s
Mean distance: 0.3167891391819771

